# "Prezzi e rendimenti con Pandas"
> "Integrare direttamente in un post un notebook di Jupyter"

- toc: false
- branch: master
- badges: true
- comments: true
- categories: [jupyter, pandas, finanza]
- image: images/2021-06-23-prezzi-e-rendimenti-con-pandas.png
- hide: false

[fastpages](https://fastpages.fast.ai/), la piattaforma sulla quale è costruito questo blog, permette di integrare direttamente dei *notebook* di `Jupyter` nei *post*. Per provare questa funzionalità, costruiamo un breve *notebook* su come gestire prezzi e rendimenti delle azioni con `Pandas`, in alternativa a Excel. In questo modo, potremo osservare in azione alcuni degli strumenti che ho descritto in un mio [post](../python-per-la-finanza/index.html) precedente.

In [1]:
import pandas as pd
import altair as alt
import yfinance as yf

Scarichiamo da Yahoo! Finance i prezzi di quattro azioni statunitensi, identificate attraverso i loro *ticker*: **Apple** (AAPL), **Microsoft** (MSFT), **McDonald's** (MCD) e **Coca-Cola** (KO). Due azioni della *new economy* e due della *old*, come si diceva qualche anno fa.

In [2]:
tickers = 'AAPL MSFT MCD KO'
data = yf.download(tickers=tickers, period='2y')

[*********************100%***********************]  4 of 4 completed


A questo punto, eliminiamo dal database che abbiamo costruito i prezzi *open*, *high*, *low* e il volume negoziato per tenerci solo i prezzi di chiusura, rettificati dei dividendi eventualmente distribuiti e delle *corporate action*. Vediamo le prime righe del *database* dei prezzi.

In [3]:
prices = data.xs('Adj Close', axis=1, level=0)
prices.tail()

,AAPL,KO,MCD,MSFT
Date,,,,
2021-06-17,131.789993,54.950001,233.880005,260.899994
2021-06-18,130.460007,53.770000,229.619995,259.429993
2021-06-21,132.300003,54.360001,232.899994,262.630005
2021-06-22,133.979996,54.560001,233.880005,265.510010
2021-06-23,133.729996,54.389999,233.725006,264.809998


Possiamo fare un primo grafico dell'andamento dei prezzi delle quattro azioni negli ultimi due anni, utilizzando `Altair`, una libreria che permette di produrre grafici interattivi.

In [4]:
alt.Chart(
    prices.reset_index().melt(
        'Date', 
        var_name='Stock', 
        value_name='Price'
    )
).mark_line().encode(
    x='Date:T',
    y='Price:Q',
    color='Stock:N',
    tooltip=['Stock', 'Price']
).properties(
    title='Andamento prezzi negli ultimi due anni'
).interactive()

alt.Chart(...)

Per avere la possibilità di confrontare i quattro grafici, ribasiamo i dati facendo partire gli andamenti da quota 100.

In [5]:
rebased_prices = prices.div(prices.iloc[0, :]).mul(100)
alt.Chart(
    rebased_prices.reset_index().melt(
        'Date', 
        var_name='Stock', 
        value_name='Price'
    )
).mark_line().encode(
    x='Date:T',
    y='Price:Q',
    color='Stock:N',
    tooltip=['Stock', 'Price']
).properties(
    title='Andamento prezzi ribasati negli ultimi due anni'
).interactive()

alt.Chart(...)

Passiamo ora ai rendimenti e calcoliamo i rendimenti mensili delle quattro azioni.

In [6]:
monthly_returns = prices.resample('M').last().pct_change()

Possiamo fare un grafico a barre dei rendimenti appena calcolati, divisi per mese.

In [7]:
alt.Chart(
    monthly_returns.reset_index().melt(
        'Date', 
        var_name='Stock', 
        value_name='Return'
    )
).mark_bar().encode(
    y='Stock:N',
    x='Return:Q',
    color='Stock:N',
    row='Date:T',
    tooltip=['Stock', 'Return']
).properties(
    title='Rendimenti percentuali per mese'
).interactive()

alt.Chart(...)

Oppure lo stesso grafico raggruppato per titolo, dove notiamo come **Apple** sia stato il titolo più volatile negli ultimi due anni, ma **Coca-Cola** e **McDonald's** abbiano avuto i maggiori *drawdown*.

In [8]:
alt.Chart(
    monthly_returns.reset_index().melt(
        'Date', 
        var_name='Stock', 
        value_name='Return'
    )
).mark_bar().encode(
    x='Date:T',
    y='Return:Q',
    color='Stock:N',
    row='Stock:N',
    tooltip=['Stock', 'Return']
).properties(
    title='Rendimenti percentuali per mese'
).interactive()

alt.Chart(...)

Ci fermiamo qui con questo post di prova, che mostra le potenzialità di analisi esplorativa di Pandas, in grado in poche righe di codice di scaricare, elaborare e visualizzare serie storiche di dati con grande facilità.